<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Grid-Search" data-toc-modified-id="Grid-Search-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Grid Search</a></span></li><li><span><a href="#Best-params-result" data-toc-modified-id="Best-params-result-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Best params result</a></span></li></ul></div>

In [39]:
nb_name = 'Classification_svm_nb_baseline-augmentation'

In [10]:
import numpy as np 
import pandas as pd
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

from clustering_utils import *
from eda_utils import *
from myutils_V6 import *


In [11]:
train, test = load_data()
# train, upsampling_info = upsampling_train(train)

train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']
combination 1 train:  ['text']
combination 2 train:  ['reply', 'reference_one']


In [12]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label
y_test = test_label

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(22166,)
(7761,)
(22166,)
(7761,)


In [13]:
set(train_label.tolist())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

# Grid Search

In [53]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
              'clf__alpha': [0.2, 0.4, 0.6, 0.8, 1]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")

clf__alpha:	0.2
tfidf__binary:	True
tfidf__min_df:	3
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.939


In [41]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
             'clf__penalty':['l2'], 'clf__C':[1,2,3]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

clf__C:	3
clf__penalty:	l2
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	None
tfidf__use_idf:	True
best f1 score: 0.992


# Best params result

In [50]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=None, binary=True, min_df=1)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)

num of words: 130526
X_train.shape (22166, 130526)
X_test.shape (7761, 130526)


In [52]:
clf = LinearSVC(penalty="l2", multi_class='ovr', C=3.0, dual=True,)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(metrics.classification_report(y_true = y_test, y_pred=pred, target_names=labels))

##########################################
## CV shows the stable result
##########################################
cv_metrics = ["precision_macro","accuracy", "f1_macro"]
cv = cross_validate(clf, X_train, y_train,scoring=cv_metrics, cv=4, return_train_score=True)
cv = pd.DataFrame(cv)
f1 = cv['test_f1_macro'].mean()
print("cv average f1 macro: ", f1)

cv

                          precision    recall  f1-score   support

             alt.atheism       0.81      0.77      0.79       319
           comp.graphics       0.76      0.81      0.78       389
 comp.os.ms-windows.misc       0.77      0.73      0.75       394
comp.sys.ibm.pc.hardware       0.74      0.75      0.75       392
   comp.sys.mac.hardware       0.83      0.86      0.84       385
          comp.windows.x       0.88      0.82      0.85       395
            misc.forsale       0.86      0.91      0.88       390
               rec.autos       0.91      0.92      0.91       395
         rec.motorcycles       0.95      0.96      0.95       398
      rec.sport.baseball       0.82      0.96      0.89       397
        rec.sport.hockey       0.99      0.89      0.94       827
               sci.crypt       0.93      0.92      0.93       396
         sci.electronics       0.80      0.74      0.77       393
                 sci.med       0.72      0.94      0.82       198
         

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro
0,6.923378,0.035001,0.997756,0.998222,0.997474,0.998075,0.997708,0.998130
1,7.074531,0.038978,0.996029,0.998165,0.995850,0.998015,0.995800,0.998074
2,7.663213,0.025434,0.989798,0.998874,0.989533,0.998797,0.989567,0.998804
3,7.899415,0.028239,0.982555,0.998821,0.983036,0.998737,0.981907,0.998813


In [44]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(metrics.classification_report(y_true = y_test, y_pred=pred, target_names=labels))


##########################################
## CV shows the stable result
##########################################
cv_metrics = ["precision_macro","accuracy", "f1_macro"]
cv = cross_validate(clf, X_train, y_train,scoring=cv_metrics, cv=4, return_train_score=True)
cv = pd.DataFrame(cv)
f1 = cv['test_f1_macro'].mean()
print("cv average f1 macro: ", f1)

cv

                          precision    recall  f1-score   support

             alt.atheism       0.78      0.84      0.81       319
           comp.graphics       0.57      0.83      0.67       389
 comp.os.ms-windows.misc       0.48      0.03      0.05       394
comp.sys.ibm.pc.hardware       0.53      0.79      0.63       392
   comp.sys.mac.hardware       0.73      0.84      0.78       385
          comp.windows.x       0.83      0.77      0.80       395
            misc.forsale       0.80      0.82      0.81       390
               rec.autos       0.87      0.93      0.89       395
         rec.motorcycles       0.92      0.96      0.94       398
      rec.sport.baseball       0.65      0.95      0.77       397
        rec.sport.hockey       1.00      0.69      0.82       827
               sci.crypt       0.90      0.92      0.91       396
         sci.electronics       0.80      0.75      0.77       393
                 sci.med       0.81      0.92      0.86       198
         

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro
0,0.098682,0.028656,0.896295,0.926074,0.887586,0.915784,0.872948,0.906472
1,0.093332,0.029897,0.927268,0.948172,0.923674,0.946282,0.919226,0.944418
2,0.091591,0.026055,0.898048,0.935585,0.887565,0.926677,0.874508,0.917221
3,0.091235,0.022755,0.895661,0.935701,0.885219,0.925173,0.871729,0.916176
